#### Step 3: Model Fine-tuning
In this notebook, you'll fine-tune the Meta Llama 2 7B large language model, deploy the fine-tuned model, and test it's text generation and domain knowledge capabilities. 

Fine-tuning refers to the process of taking a pre-trained language model and retraining it for a different but related task using specific data. This approach is also known as transfer learning, which involves transferring the knowledge learned from one task to another. Large language models (LLMs) like Llama 2 7B are trained on massive amounts of unlabeled data and can be fine-tuned on domain domain datasets, making the model perform better on that specific domain.

Input: A train and an optional validation directory. Each directory contains a CSV/JSON/TXT file.
For CSV/JSON files, the train or validation data is used from the column called 'text' or the first column if no column called 'text' is found.
The number of files under train and validation should equal to one.

- **You'll choose your dataset below based on the domain you've chosen**

Output: A trained model that can be deployed for inference.\
After you've fine-tuned the model, you'll evaluate it with the same input you used in project step 2: model evaluation. 

---

#### Set up

---
Install and import the necessary packages. Restart the kernel after executing the cell below. 

---

In [6]:
!pip install --upgrade sagemaker datasets

Select the model to fine-tune

In [1]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In the cell below, choose the training dataset text for the domain you've chosen and update the code in the cell below:  

To create a finance domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/finance"`

To create a medical domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/medical"`

To create an IT domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/it"`

In [11]:
from sagemaker.jumpstart.estimator import JumpStartEstimator
import boto3

estimator = JumpStartEstimator(model_id=model_id,  environment={"accept_eula": "true"},instance_type = "ml.g5.2xlarge") 

estimator.set_hyperparameters(instruction_tuned="False", epoch="5")

#Fill in the code below with the dataset you want to use from above 
#example: estimator.fit({"training": f"s3://genaiwithawsproject2024/training-datasets/finance"})
estimator.fit({"training": f"s3://genaiwithawsproject2024/training-datasets/it"})

INFO:sagemaker:Creating training-job with name: meta-textgeneration-llama-2-7b-2024-04-04-13-04-49-611


2024-04-04 13:04:49 Starting - Starting the training job...
2024-04-04 13:05:05 Pending - Training job waiting for capacity...
2024-04-04 13:05:20 Pending - Preparing the instances for training...
2024-04-04 13:05:55 Downloading - Downloading input data...........................
2024-04-04 13:10:36 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-04-04 13:10:38,430 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-04-04 13:10:38,449 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-04 13:10:38,458 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-04-04 13:10:38,461 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-04-04 13:10:46,527 sagemaker-training-toolkit INFO     Installing depende

#### Deploy the fine-tuned model
---
Next, we deploy the domain fine-tuned model. We will compare the performance of the fine-tuned and pre-trained model.

---

In [12]:
finetuned_predictor = estimator.deploy()

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.
INFO:sagemaker:Creating model with name: meta-textgeneration-llama-2-7b-2024-04-04-13-20-38-816
INFO:sagemaker:Creating endpoint-config with name meta-textgeneration-llama-2-7b-2024-04-04-13-20-38-808
INFO:sagemaker:Creating endpoint with name meta-textgeneration-llama-2-7b-2024-04-04-13-20-38-808


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'ml.g5.2xlarge for endpoint usage' is 2 Instances, with current utilization of 2 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

#### Evaluate the pre-trained and fine-tuned model
---
Next, we use the same input from the model evaluation step to evaluate the performance of the fine-tuned model and compare it with the base pre-trained model. 

---

Create a function to print the response from the model

In [13]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response}")
    print("\n==================================\n")

Now we can run the same prompts on the fine-tuned model to evaluate it's domain knowledge.   

**Replace "inputs"** in the next cell with the input to send the model based on the domain you've chosen. 

**For financial domain:**

  "inputs": "Replace with sentence below from text"  
- "The  investment  tests  performed  indicate"
- "the  relative  volume  for  the  long  out  of  the  money  options, indicates"
- "The  results  for  the  short  in  the  money  options"
- "The  results  are  encouraging  for  aggressive  investors"

**For medical domain:** 

  "inputs": "Replace with sentence below from text" 
- "Myeloid neoplasms and acute leukemias derive from"
- "Genomic characterization is essential for"
- "Certain germline disorders may be associated with"
- "In contrast to targeted approaches, genome-wide sequencing"

**For IT domain:** 

  "inputs": "Replace with sentence below from text" 
- "Traditional approaches to data management such as"
- "A second important aspect of ubiquitous computing environments is"
- "because ubiquitous computing is intended to" 
- "outline the key aspects of ubiquitous computing from a data management perspective."

In [15]:
payload = {
    "inputs": "Traditional approaches to data management such as",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

Traditional approaches to data management such as
> [{'generated_text': " data warehousing and data virtualization are no longer adequate to support the real-time requirements of modern enterprises.\nIn this webinar, we'll introduce the concept of a data lake, a new paradigm for data management that is emerging to meet the demands of"}]




Do the outputs from the fine-tuned model provide domain-specific insightful and relevant content? You can continue experimenting with the inputs of the model to test it's domain knowledge. 

**Use the output from this notebook to fill out the "model fine-tuning" section of the project documentation report**

**After you've filled out the report, run the cells below to delete the model deployment** 

`IF YOU FAIL TO RUN THE CELLS BELOW YOU WILL RUN OUT OF BUDGET TO COMPLETE THE PROJECT`

In [ ]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()